In [ ]:
with open(".env", "w") as f:
    f.write("GMAIL_USER=qochiwam@gmail.com\n")
    f.write("GMAIL_PASSWORD=yvst fkqa odnl qcib\n")

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import time
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import urllib3
from urllib.parse import urljoin
from flask import Flask, request
from dotenv import load_dotenv


load_dotenv()

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

app = Flask(__name__)

# ---- Configuration ----
GMAIL_USER = os.getenv("GMAIL_USER")
GMAIL_PASSWORD = os.getenv("GMAIL_PASSWORD")  
TO_EMAIL = GMAIL_USER  

BANK_WEBSITES = [
    "https://www.letshego.com/botswana",
    "https://www.centralbank.org.sz/tenders/",
    "https://www.sbs.co.sz/tenders/",
    "https://www.fincorp.co.sz/tenders/",
    "https://www.idce.co.sz",
    "https://www.centralbank.org.ls",
    "https://www.lpb.co.ls/tenders/",
    "https://www.fdh.co.mw",
    "https://www.fdh.co.mw/fdhfinancialholdings",
    "https://www.natbank.co.mw",
    "https://www.nbs.mw",
    "https://www.bankbic.na/",
    "https://www.bon.com.na/",
    "https://www.capricorn.com.na/Pages/default.aspx",
    "https://www.dbn.com.na/",
    "https://www.smebank.com.na/",
    "https://www.dbs.sc",
    "https://www.bankalhabib.com",
    "https://www.scb.sc",
    "https://www.abbank.co.zm/",
    "https://www.finca.co.zm/",
    "https://www.firstalliancebankzambia.com/",
    "https://www.izb.co.zm/",
    "https://www.izwezambia.com/",
    "https://www.mfinance.co.zm/",
    "https://www.natsave.co.zm/",
    "https://www.boz.zm/",
    "https://www.ubazambia.com/",
    "https://www.zanaco.co.zm/",
    "https://www.afrasiabank.com/",
    "https://www.bancabc.co.zw/",
    "https://www.afcholdings.co.zw/",
    "https://www.cabs.co.zw/",
    "https://www.firstcapitalbank.co.zw/",
    "https://www.cbz.co.zw/",
    "https://www.africancentury.co.zw/",
    "https://ecobank.com/tg/procurement",
    "https://www.fbc.co.zw/",
    "https://www.fmbcapitalgroup.com/",
    "https://www.brightzim.com/",
    "https://nmbz.co.zw/",
    "https://www.rbz.co.zw/index.php/procurement",
    "https://www.zb.co.zw/",
    "https://www.idbz.co.zw/portal/",
]

# Tender keywords (finance-specific)
TENDER_KEYWORDS = [
    "bid", "bond valuation", "proposal", "RFP", "request for proposal",
    "ECL model", "automation", "IFRS 9", "credit risk", "discount rate",
    "financial services", "data analytics", "risk management", "valuation", "stress testing"
]

# ---- Scraping Function ----
def check_tenders():
    found_tenders = []

    for url in BANK_WEBSITES:
        try:
            max_retries = 3
            timeout = 30
            for attempt in range(max_retries):
                try:
                    response = requests.get(url, timeout=timeout, verify=False)
                    soup = BeautifulSoup(response.text, "html.parser")
                    break
                except requests.exceptions.RequestException as e:
                    if attempt == max_retries - 1:
                        print(f"Error scraping {url} after {max_retries} attempts: {e}")
                        continue
                    time.sleep(2)

            # Look for tenders inside <a> links, <h2>, <p>, or table rows
            tender_texts = soup.find_all(["a", "h2", "p", "li", "td"])  
            for tag in tender_texts:
                text = tag.get_text(strip=True).lower()
                if any(keyword in text for keyword in TENDER_KEYWORDS):
                    tender_link = tag.get("href")
                    
                    # Ensure tender_link is a full URL
                    if tender_link:
                        tender_link = urljoin(url, tender_link)
                    else:
                        tender_link = "No link available"
                    
                    found_tenders.append(f"{url}: {text} ({tender_link})")

        except Exception as e:
            print(f"Error scraping {url}: {e}")

    # Send email only if tenders are found
    if found_tenders:
        content = "\n".join(found_tenders)
        send_email("🚀 New Relevant Bank Tenders Found!", content)
    else:
        print("No relevant tenders found.")

# ---- Email Function ----
def send_email(subject, content):
    msg = MIMEMultipart()
    msg["From"] = GMAIL_USER
    msg["To"] = TO_EMAIL
    msg["Subject"] = subject
    msg.attach(MIMEText(content, "plain"))

    try:
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()
        server.login(GMAIL_USER, GMAIL_PASSWORD)
        server.send_message(msg)
        server.quit()
        print("✅ Email sent successfully!")
    except Exception as e:
        print(f"❌ Failed to send email: {e}")

# ---- Flask route to trigger the check_tenders function ----
@app.route('/')
def run_script():
    check_tenders()
    return """
    <h1>Tenders check completed!</h1>
    <p>To find your permanent URL:</p>
    <ol>
        <li>Look for the "Open in new tab" button in the top right of the Webview</li>
        <li>Click it to see your permanent URL in the new tab's address bar</li>
    </ol>
    """

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)  # Changed port to 5000 for better compatibility
